In [2]:
#importing alpacas
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [3]:
#setting random number generator
np.random.seed(42)


In [6]:
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

#setting and confirming keys 
alpaca_api_key = 'PK74K7VZ597PLHX1JV0Q'
alpaca_secret_key = 'bncnRyrcNLt150gOYddntIUaEoYWENnrbxeFaDBn'
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v1')

ModuleNotFoundError: No module named 'dotenv'

In [5]:
#SPY = stock, AGG = bonds 
# Set the ticker
list_of_tickers = ['SPY','AGG']

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical closing data in data frame  data 
stk_bnd_df = api.get_barset(
    list_of_tickers,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

stk_bnd_df = stk_bnd_df.droplevel(axis=1, level=0)
stk_bnd_df.head()
stk_bnd_df.drop(columns=['open', 'high', 'low', 'volume'],inplace = True)
stk_bnd_df.index = stk_bnd_df.index.date
stk_bnd_df.columns = ['SPY','AGG']
stk_bnd_df.head()
                        

APIError: access key verification failed (Code = 40110000)

In [ ]:
#daily roi
daily_returns = stk_bnd_df.pct_change()
daily_returns.head()

#need this for montecarlo
avg_daily_return_SPY = daily_returns.mean()['SPY']
avg_daily_return_AGG =daily_returns.mean()['AGG'] 


In [ ]:
#calculating volatility (std)
std_dev_daily_return_SPY = daily_returns.std()['SPY']
std_dev_daily_return_AGG = daily_returns.std()['AGG']

#volatility for SPY
std_dev_daily_return_SPY 


In [ ]:
#volatlity for AGG
std_dev_daily_return_AGG

In [ ]:
#last days closing prices
last_days = stk_bnd_df.iloc[-1]
SPY_LAST_CLOSING= last_days['SPY']
AGG_LAST_CLOSING = last_days['AGG']

In [ ]:
# Setup the Monte Carlo Parameters
number_simulations = 500
number_records = 252 * 30

#portfolio weights
weights = [0.60,0.40]

#empty df for simulation
simulated_price_df = pd.DataFrame
portfolio_cumulative_returns = pd.DataFrame

# Run the simulation of projecting stock prices for the next trading year, `1000` times
for n in range(num_simulations):

    # Initialize the simulated prices list with the last closing price of `JNJ` and `MU`
    simulated_spy_prices = [SPY_LAST_CLOSING]
    simulated_mu_prices = [AGG_LAST_CLOSING]
    
    # Simulate the returns for 252 days
    for i in range(num_trading_days):
        
        # Calculate the simulated price using the last price within the list
        simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_return_SPY, std_dev_daily_return_SPY))
        simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_return_AGG, std_dev_daily_return_AGG))
        
        # Append the simulated price to the list
        simulated_spy_prices.append(simulated_spy_price)
        simulated_agg_prices.append(simulated_agg_price)
    
    # Append a simulated prices of each simulation to DataFrame
    simulated_price_df["SPY prices"] = pd.Series(simulated_spy_prices)
    simulated_price_df["AGG prices"] = pd.Series(simulated_agg_prices)
    
    # Calculate the daily returns of simulated prices
    simulated_daily_returns = simulated_price_df.pct_change()
    

    # Use the `dot` function with the weights to multiply weights with each column's simulated daily returns
    portfolio_daily_returns = simulated_daily_returns.dot(weights)
    
    # Calculate the normalized, cumulative return series
    portfolio_cumulative_returns[n] = (1 + portfolio_daily_returns.fillna(0)).cumprod()

# Print records from the DataFrame
portfolio_cumulative_returns.head()


In [ ]:
#500 simulations over 30 years 
plot_title = f"{n+1} Simulations of Cumulative Return Trajectories over the next 30 years"
portfolio_cumulative_returns.plot(title=plot_title)

In [ ]:
#last row for cumulative returns at 30 years 
30years_returns =portfolio_cumulative_returns.iloc[-1,:]
30years_returns = 30_years_returns.max().max()


In [ ]:
#select the last row for cumulative returns at 20 years 
twentyyearsreturn = portfolio_cumulative_returns.iloc[20*252,:]
twentyyearsreturn.max()

In [ ]:
#confidence interval
confidence_ = twentyyearsreturn.quantile(q=[0.05,0.95])
confidence_

In [ ]:
#distribution of ending returns
plt.figure();
twentyyearsreturn.plot(kind='hist',density=True,bins=10)
pltaxvline(confidence_.iloc[0],color='y')
plt.axvline(confidence_.iloc[1],color='y')


In [ ]:
#What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
confidence_30yrs_10th = 30years_returns.quantile(q=[0.90,0.10]) 
confidence_30yrs_50th = 30years_returns.quantile(q=[0.50,0.50])
confidence_30yrs_90th = 30years_returns.quantile(q=[0.10,0.90])

confidence_30yrs_10th
confidence_30yrs_50th
confidence_30yrs_90th

In [ ]:
# Set initial investment, predict returns at different confidence intervals 
initial_investment = 20000


confidence_30yrs_10th_withnewinvestment = initial_investment * confidence_30yrs_10th
confidence_30yrs_50th_withnewinvestment = initial_investment * confidence_30yrs_50th
confidence_30yrs_90th_withnewinvestment = initial_investment * confidence_30yrs_90th

confidence_30yrs_10th_withnewinvestment

confidence_30yrs_50th_withnewinvestment

confidence_30yrs_90th_withnewinvestment

In [ ]:
 #Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
    
    #It would exceed. 

In [7]:
#How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?
#Likely increase by a hefty value, from $20,000 to $30,000 is alot. 